In [ ]:
rm(list = ls())
library("dplyr")
library("plm")
library("AER")
set.seed(1313)

In [7]:
get_data <- function(N, TT, alpha, beta){
  
  df <- setNames(data.frame(matrix(0, ncol = 4, nrow = N*(TT+10))), c("j", "t", "y", "x"))
  aux <- 1
  
  for (j in seq(1:N)){
    
    start <- j
    y <- 0 
    x <- rnorm(1,0,0.9)
    
    for(t in seq(1:(TT+10))){
      
      c <- rnorm(1,0,1)
      u <- rnorm(1,0,1)
      v <- rnorm(1,0,0.9)
      x <- (0.8)*x + v 
      y <- (alpha)*y + (beta)*x + c + u
      df[aux, ] <- c(j, t, y, x)
      aux <- aux + 1
      
    }
    
  }
  
  df <- df %>% group_by(j) %>% 
    mutate(yL1 = dplyr::lag(y), 
           yL2 = dplyr::lag(yL1),
           xL1 = dplyr::lag(x), 
           xL2 = dplyr::lag(xL1),
           dif_y = c(NA, diff(y)), 
           dif_yL1 = c(NA, diff(yL1)), 
           dif_x = c(NA, diff(x)), 
           dif_xL1 = c(NA, diff(xL1))) %>% 
    do(tail(., n = 10))
  
  return(df)
  
}

get_LSDV <- function(data, N, TT, alpha) {
  
  df <- setNames(data.frame(matrix(0, ncol = 7, nrow = 1)),
                 c("estimador", "alpha", "st_alpha",
                   "beta", "st_beta", "RMSE", "tamano"))
  
  LSDV <- plm(y ~ yL1 + x  -1, 
              data = data, 
              method = "within", 
              effect = "individual")
  
  pred <- data$yL1*(summary(LSDV)$coefficients[1,1]) +
    data$x*(summary(LSDV)$coefficients[2,1])
  
  t <- (summary(LSDV)$coefficients[1,1] - alpha)/summary(LSDV)$coefficients[1,2]
  if ((pt(abs(t), (N*TT-2), lower.tail = FALSE, log.p = FALSE)*2) < 0.05){
    tamano_LSDV <- 1
  } else { 
    tamano_LSDV <- 0
  }
  
  
  df[1, ] <- c("LSDV", 
               summary(LSDV)$coefficients[1,1],
               summary(LSDV)$coefficients[1,2],
               summary(LSDV)$coefficients[2,1],
               summary(LSDV)$coefficients[2,2],
               sqrt(mean((data$y - pred)^2)), 
               tamano_LSDV)
  return(df)
  
}

get_AB1 <- function(data, N, TT, alpha) { 

  
  pdata <- pdata.frame(data, index = c("j", "t"))
  
  df <- setNames(data.frame(matrix(0, ncol = 7, nrow = 1)),
                 c("estimador", "alpha", "st_alpha",
                   "beta", "st_beta", "RMSE", "tamano"))
  
  AB1 <- pgmm(y ~ lag(y, 1) + x |
                lag(y, 2:99),
              data = pdata,
              effect = "individual",
              model = "onestep",
              transformation = "d",
              collapse = FALSE)
  
  pred <- data$yL1*(summary(AB1)$coefficients[1,1]) +
    data$x*(summary(AB1)$coefficients[2,1])
  
  t <- (summary(AB1)$coefficients[1,1] - alpha)/summary(AB1)$coefficients[1,2]
  if ((pt(abs(t), (N*TT-2), lower.tail = FALSE, log.p = FALSE)*2) < 0.05){
    tamano_AB1 <- 1
  } else { 
    tamano_AB1 <- 0
  }

  
  df[1, ] <- c("AB1", 
               summary(AB1)$coefficients[1,1],
               summary(AB1)$coefficients[1,2],
               summary(AB1)$coefficients[2,1],
               summary(AB1)$coefficients[2,2],
               sqrt(mean((data$y - pred)^2)), 
               tamano_AB1)
  return(df)
  
}

get_AB2 <- function(data, N, TT, alpha) { 
  
  
  pdata <- pdata.frame(data, index = c("j", "t"))
  
  df <- setNames(data.frame(matrix(0, ncol = 7, nrow = 1)),
                 c("estimador", "alpha", "st_alpha",
                   "beta", "st_beta", "RMSE", "tamano"))
  
  AB2 <- pgmm(y ~ lag(y, 1) + x |
                lag(y, 2:99),
              data = pdata,
              effect = "individual",
              model = "twosteps",
              transformation = "d",
              collapse = FALSE)
  
  pred <- data$yL1*(summary(AB2)$coefficients[1,1]) +
    data$x*(summary(AB2)$coefficients[2,1])
  
  t <- (summary(AB2)$coefficients[1,1] - alpha)/summary(AB2)$coefficients[1,2]
  if ((pt(abs(t), (N*TT-2), lower.tail = FALSE, log.p = FALSE)*2) < 0.05){
    tamano_AB2 <- 1
  } else { 
    tamano_AB2 <- 0
  }
  
  df[1, ] <- c("AB2", 
               summary(AB2)$coefficients[1,1],
               summary(AB2)$coefficients[1,2],
               summary(AB2)$coefficients[2,1],
               summary(AB2)$coefficients[2,2],
               sqrt(mean((data$y - pred)^2)), 
               tamano_AB2)
  return(df)
  
}

get_BB1 <- function(data, N, TT, alpha) {
  
  pdata <- pdata.frame(data, index = c("j", "t"))
  df <- setNames(data.frame(matrix(0, ncol = 7, nrow = 1)), 
                 c("estimador", "alpha", "st_alpha",
                   "beta", "st_beta", "RMSE", "tamano"))
  
  BB1 <- pgmm(y ~ lag(y, 1) + x | 
                lag(y, 2:99) + lag(x, 1:99),
              data = pdata, 
              effect = "individual", 
              model = "onestep", 
              transformation = "ld",
              collapse = FALSE)
  
  pred <- data$yL1*(summary(BB1)$coefficients[1,1]) + 
    data$x*(summary(BB1)$coefficients[2,1])
  
  t <- (summary(BB1)$coefficients[1,1] - alpha)/summary(BB1)$coefficients[1,2]
  if ((pt(abs(t), (N*TT-2), lower.tail = FALSE, log.p = FALSE)*2) < 0.05){
    tamano_BB1 <- 1
  } else { 
    tamano_BB1 <- 0
  }
  

  df[1, ] <- c("BB1", 
               summary(BB1)$coefficients[1,1],
               summary(BB1)$coefficients[1,2],
               summary(BB1)$coefficients[2,1],
               summary(BB1)$coefficients[2,2],
               sqrt(mean((data$y - pred)^2)), 
               tamano_BB1)
  return(df)
  
}

get_AH <- function(data, N, TT, alpha) {
  
  tamano_AH <- 0
  
  df <- setNames(data.frame(matrix(0, ncol = 7, nrow = 1)), 
                 c("estimador", "alpha", "st_alpha",
                   "beta", "st_beta", "RMSE", "tamano"))
  
  pdata <- pdata.frame(data, index = c("j", "t"))
  
  AH <- ivreg(dif_y ~ dif_yL1 + dif_x - 1 | . - dif_yL1 + yL2, 
              data = pdata)
  
  t <- (summary(AH)$coefficients[1,1] - alpha)/summary(AH)$coefficients[1,2]
  if ((pt(abs(t), (N*TT-2), lower.tail = FALSE, log.p = FALSE)*2) < 0.05){
    tamano_AH <- 1
  } else { 
    tamano_AH <- 0
  }
  
  df[1, ] <- c("AH",
               summary(AH)$coefficients[1,1],
               summary(AH)$coefficients[1,2],
               summary(AH)$coefficients[2,1],
               summary(AH)$coefficients[2,2],
               sqrt(mean((data$y - predict(AH))^2)), 
               tamano_AH)
  return(df)
  
}

get_kiviet <- function(data, N, TT, alpha){
  
  tamano_kiviet <- 0
  df <- setNames(data.frame(matrix(0, ncol = 7, nrow = 1)), 
                 c("estimador", "alpha", "st_alpha",
                   "beta", "st_beta", "RMSE", "tamano"))
  pdata <- pdata.frame(data, index = c("j", "t"))
  kiviet <-lm(y ~ yL1 + x - 1, data=pdata)
  alpha_hat <- kiviet$coefficients[1]
  beta_hat <- kiviet$coefficients[2]
  N <- nrow(data %>% group_by(j) %>% summarise())
  t <- nrow(data %>% group_by(t) %>% summarise())
  NT <- N*t
  k <- 2
  
  within_data <- data %>% group_by(j) %>%
    transmute(ddot_y = y - mean(y, na.rm = TRUE), 
              ddot_yL1 = yL1 - mean(yL1, na.rm = TRUE), 
              ddot_x = x - mean(x, na.rm = TRUE)) %>% 
    do(na.omit(.))
  Z <- append(as.vector(within_data$ddot_yL1), 
              as.vector(within_data$ddot_x))
  u_hat <- within_data$ddot_y - 
    within_data$ddot_yL1 * alpha_hat - 
    within_data$ddot_x * beta_hat
  SRC <- t(u_hat)%*%u_hat
  sigma2e <- SRC/(NT-N-t-k+1)
  se_a <- sqrt(solve(t(Z)%*%Z)*sigma2e)
  se_b <- sqrt(diag(vcov(kiviet)))[2]
  
  t <- as.numeric((alpha_hat-alpha)/se_a)
  if((pt(abs(t), (NT-k), lower.tail = FALSE, log.p = FALSE)*2) < 0.05){
    tamano_kiviet <- tamano_kiviet + 1
  }
  
  pred <- data$y*alpha_hat + data$x*beta_hat
  
  df[1, ] <- c("kiviet", 
               alpha_hat, 
               se_a, 
               beta_hat, 
               se_b, 
               sqrt(mean((data$y - pred)^2)), 
               tamano_kiviet)
  return(df)
} 

montecarlo <- function(simulations, N, TT, alpha, beta){
  
  temp <- list()
  
  for (n in seq(1:simulations)) {
    
    #print(n)
    data <- get_data(N, TT, alpha, beta)
    df <- rbind(get_LSDV(data, N, TT, alpha), 
                get_AB1(data, N, TT, alpha), 
                get_AB2(data, N, TT, alpha),
                get_BB1(data, N, TT, alpha), 
                get_AH(data, N, TT, alpha), 
                get_kiviet(data, N, TT, alpha))
    temp[[n]] <- df 
    
  }
  
  res <- dplyr::bind_rows(temp)
  res <- res %>% mutate(across(!estimador, as.numeric)) %>%  
    group_by(estimador) %>% summarise(alpha_hat = mean(alpha), 
                                      st_alpha = mean(st_alpha), 
                                      beta_hat = mean(beta), 
                                      st_beta = mean(st_beta), 
                                      RMSE = mean(RMSE), 
                                      tamano = mean(tamano))
  
  return(res)

}

In [8]:
EJ1 <- montecarlo(1000, 30, 10, 0.5, 0)
print(EJ1)

# A tibble: 6 × 7
  estimador alpha_hat st_alpha beta_hat st_beta  RMSE tamano
  <chr>         <dbl>    <dbl>    <dbl>   <dbl> <dbl>  <dbl>
1 AB1           0.439   0.0761 -0.00410  0.126  1.43   0.146
2 AB2           0.435   0.0931 -0.00298  0.149  1.43   0.088
3 AH            0.501   0.142  -0.00357  0.123  1.63   0.009
4 BB1           0.485   0.0582 -0.00681  0.0736 1.41   0.088
5 kiviet        0.495   0.0481 -0.00559  0.0551 0.819  0.064
6 LSDV          0.333   0.0575 -0.00656  0.0804 1.44   0.834


In [9]:
EJ2 <- montecarlo(1000, 50, 10, 0.5, 0)
print(EJ2)

# A tibble: 6 × 7
  estimador alpha_hat st_alpha  beta_hat st_beta  RMSE tamano
  <chr>         <dbl>    <dbl>     <dbl>   <dbl> <dbl>  <dbl>
1 AB1           0.463   0.0610 -0.00159   0.102  1.42   0.103
2 AB2           0.462   0.0672 -0.00147   0.112  1.42   0.089
3 AH            0.500   0.110  -0.00173   0.0947 1.63   0.011
4 BB1           0.492   0.0484  0.00169   0.0636 1.41   0.058
5 kiviet        0.498   0.0369  0.000165  0.0425 0.817  0.059
6 LSDV          0.335   0.0445 -0.000380  0.0618 1.44   0.968


In [10]:
EJ3 <- montecarlo(1000, 30, 20, 0.8, 0)
print(EJ3)

# A tibble: 6 × 7
  estimador alpha_hat st_alpha  beta_hat st_beta  RMSE tamano
  <chr>         <dbl>    <dbl>     <dbl>   <dbl> <dbl>  <dbl>
1 AB1           0.689   0.0891  0.00167   0.124  1.46   0.252
2 AB2           0.685   0.106   0.000999  0.145  1.46   0.175
3 AH            0.810   0.251   0.00214   0.125  2.35   0.001
4 BB1           0.782   0.0473 -0.00268   0.0752 1.41   0.084
5 kiviet        0.796   0.0441 -0.00227   0.0551 0.477  0.021
6 LSDV          0.575   0.0499  0.000411  0.0783 1.51   0.993


In [11]:
EJ4 <- montecarlo(1000, 30, 20, 0.92, 0)
print(EJ4)

# A tibble: 6 × 7
  estimador alpha_hat st_alpha beta_hat st_beta  RMSE tamano
  <chr>         <dbl>    <dbl>    <dbl>   <dbl> <dbl>  <dbl>
1 AB1           0.717   0.112   0.00257  0.120  1.65   0.426
2 AB2           0.711   0.133   0.00432  0.140  1.67   0.332
3 AH            0.927   0.441   0.00213  0.130  3.56   0    
4 BB1           0.907   0.0357  0.00428  0.0744 1.42   0.084
5 kiviet        0.917   0.0420  0.00288  0.0554 0.296  0.003
6 LSDV          0.662   0.0456  0.00288  0.0769 1.69   1    


In [12]:
EJ5 <- montecarlo(1000, 50, 30, 0.5, 0)
print(EJ5)

# A tibble: 6 × 7
  estimador alpha_hat st_alpha  beta_hat st_beta  RMSE tamano
  <chr>         <dbl>    <dbl>     <dbl>   <dbl> <dbl>  <dbl>
1 AB1           0.464   0.0615  0.00337   0.102  1.42   0.1  
2 AB2           0.463   0.0673  0.00320   0.111  1.42   0.082
3 AH            0.500   0.110   0.00440   0.0947 1.63   0.008
4 BB1           0.491   0.0487 -0.000344  0.0632 1.41   0.053
5 kiviet        0.499   0.0369  0.000350  0.0423 0.815  0.049
6 LSDV          0.336   0.0444  0.000797  0.0616 1.44   0.971


In [13]:
EJ6 <- montecarlo(1000, 100, 7, 0.8, 1)
print(EJ6)

# A tibble: 6 × 7
  estimador alpha_hat st_alpha beta_hat st_beta  RMSE tamano
  <chr>         <dbl>    <dbl>    <dbl>   <dbl> <dbl>  <dbl>
1 AB1           0.772   0.0440    1.01   0.0674  1.44  0.08 
2 AB2           0.775   0.0511    0.988  0.0754  1.45  0.063
3 AH            0.805   0.236     1.00   0.0957  5.56  0    
4 BB1           0.798   0.0146    1.01   0.0450  1.41  0.063
5 kiviet        0.799   0.0155    1.00   0.0368  1.04  0.002
6 LSDV          0.722   0.0166    1.04   0.0446  1.47  0.998


In [14]:
EJ7 <- montecarlo(1000, 100, 4, 0.8, 0)
print(EJ7)

# A tibble: 6 × 7
  estimador alpha_hat st_alpha  beta_hat st_beta  RMSE tamano
  <chr>         <dbl>    <dbl>     <dbl>   <dbl> <dbl>  <dbl>
1 AB1           0.760   0.0535 -0.00246   0.0741 1.42   0.114
2 AB2           0.758   0.0624 -0.00287   0.0822 1.43   0.106
3 AH            0.801   0.140  -0.00211   0.0673 2.30   0    
4 BB1           0.792   0.0319  0.000519  0.0485 1.42   0.061
5 kiviet        0.798   0.0241  0.000386  0.0302 0.468  0.02 
6 LSDV          0.573   0.0275  0.000625  0.0429 1.51   1    
